In [1]:
from pathlib import Path
from sqlalchemy import create_engine

import pandas as pd

from iacele import costo_ventas_mes_func

# Se define la ruta para la base de datos local SQLite
project_name = Path.cwd().parent.parent.name
data_projetc_path = Path.home().joinpath(f'Dropbox/La Casa Del Carpintero/Departamento de Programación/data_projects_git/data_{project_name}')
db_file = 'comisiones.db'
db_file_path_str = str(data_projetc_path.joinpath(db_file))

# Se crea el objeto engine para trabajarlo con los metodos de pandas
engine = create_engine(f'sqlite:///{db_file_path_str}')

In [ ]:
for mes in [10]:
# for i in range(8):
    # mes = i+1
    
    print(f'\nSe inicia función para el mes {mes}')

    costo_ventas_after_comisiones_clientes, especiales_sin_usar, ultimo_costo = costo_ventas_mes_func(mes, 'local')


    if type(costo_ventas_after_comisiones_clientes) == pd.DataFrame and type(especiales_sin_usar) == pd.DataFrame and type(ultimo_costo) == pd.DataFrame:

        with engine.connect() as conn, conn.begin():
            costo_ventas_after_comisiones_clientes.to_sql(name = f'costo_ventas_{mes}_2024', index= False, con = conn, if_exists = 'replace')
            ultimo_costo.to_sql(name = f'ultimo_costo_{mes}_2024', index= False, con = conn, if_exists = 'replace')
            especiales_sin_usar.to_sql(name = f'compras_especiales_sin_usar_{mes}_2024', index= False, con = conn, if_exists = 'replace')

        engine.dispose()

        
        costo_ventas_after_comisiones_clientes['short_salesperson_name'] = costo_ventas_after_comisiones_clientes['salesperson_name'].str.split(" ").str.get(0)
        excel_data_drive_comisiones = costo_ventas_after_comisiones_clientes[[
                                'name',
                                'short_salesperson_name',
                                'sale_team_description',
                                'business_model',
                                'utilidad_partida_$_after_cc',
                            ]]

        
        excel_ventas_name = f'ventas_{mes}_2024.xlsx'
        costo_ventas_after_comisiones_clientes.to_excel(str(Path.home().joinpath(f'desktop/{excel_ventas_name}')))
        print(f'\nSe escribió con éxito el archivo "{excel_ventas_name}" en el escritorio')
        
        excel_data_drive_comisiones_name = f'data_drive_comisiones_{mes}_2024.xlsx'
        excel_data_drive_comisiones.to_excel(str(Path.home().joinpath(f'desktop/{excel_data_drive_comisiones_name}')))
        print(f'\nSe escribió con éxito el archivo "{excel_data_drive_comisiones_name}" en el escritorio')

        print('\nSe registró con éxito en la base de datos las siguientes tablas:')
        print(f'   -> costo_ventas_{mes}_2024')
        print(f'   -> ultimo_costo_{mes}_2024' )
        print(f'   -> compras_especiales_sin_usar_{mes}_2024')

    else:
        print(f'Fallaron algunos checks. No se guardó nada en la base de datos.')